# Objetivo
Este script es el primero de una serie de scripts utilizados en el TFM para crear un corpus adicional de notas con el que enriquecer el conjunto de entrenamiento.

Como input, lee todas las notas de diversos corpus.
Como output, devuelve el identificador de las notas que debemos anotar

**Primer objetivo del script**

Del conjunto de notas de los distintos corpus (BARR2, CANTEMIST, CODIESP, IULA, LIVINGNER, MEDDOCAN, NUBes, PHARMACONER, PROFNER) vamos a seleccionar sólo aquellas probables de tener información relativa a ocupación.

Para ello aplicamos un gazetteer y un conjunto de reglas basadas en expresiones regulares. Todas aquellas notas candidatas a tener ocupaciones las anotamos.

**Segundo objetivo del script**

Eliminamos las notas que están duplicadas (en un mismo corpus puede haber notas en las carpetas de train, development y test que también estén en la carpeta background)

In [ ]:
# Cargamos las librerías necesarias
import numpy as np
import pandas as pd
import re
import os, glob
from sklearn.datasets import load_files

# Habilitamos display interactivo y enlace con nuestra cuenta de google
from google.colab import data_table
from google.colab import drive

In [ ]:
# Enlazamos nuestro notebook en Colab con nuestro almacenamiento en Google Drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Vemos la estructura de los archivos
#for dirname, _, filenames in os.walk('/content/drive/MyDrive/Colab Notebooks/NUBes'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [ ]:
# Indicamos la ruta en la que están los archivos
#DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Corpus/BARR2"
#DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Corpus/CANTEMIST"
#DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Corpus/CODIESP"
#DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Corpus/IULA" # NO LO VAMOS A USAR
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Corpus/LIVINGNER"
#DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOCAN"
#DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Corpus/NUBes"
#DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Corpus/PHARMACONER"
#DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Corpus/PROFNER" # YA ESTÁ ANOTADO CON OCUPACIONES

# Los corpus de MEDDOCAN y SHAC al tener anotaciones para ocupación ya nos permite tener localizadas las notas con ocupaciones
# (si juntamos todos los .ann procesados de MEDDOCAN y SHAC y cogemos el nombre del fichero sería equivalente a hacer esto)

# Obviamente este código no debemos ejecutarlo sobre MEDDOPROF

Para poder usar satisfactoriamente la función `load_files`, organizamos el conjunto de notas de cada corpus en carpetas. Por ejemplo para el corpus BARR2 organizamos las carpetas en `train`, `test`, `dev` y `background`


In [ ]:
# Para poder usar satisfactoriamente la función load_files, organizamos el conjunto de datos en carpetas
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html
data = load_files(DATA_DIR, encoding="utf-8", decode_error="replace", shuffle=False)

In [ ]:
# Vemos el contenido de data
print(dir(data))
# Vemos la longitud de data, para comprobar si se han cargado bien todos los archivos .txt
len(data.data)

['DESCR', 'data', 'filenames', 'target', 'target_names']


14972

In [ ]:
# Creamos un df, con el texto y con el nombre del fichero que lo contiene
df = pd.DataFrame(data.data, columns = ['texto'])
df['fichero'] = data.filenames
#df.head(4)

# En el caso de que haya ficheros que no queramos (con otra extensión como .ANN o de dos subtareas distintas) filtrar con str.contains()
df = df[df['fichero'].str.contains(".txt")]
df.head(4)

,texto,fichero
0,"El 2 de febrero de 2020, una mujer de 28 años ...",/content/drive/MyDrive/Colab Notebooks/Corpus/...
1,Ingresó en nuestro hospital un hombre de 23 añ...,/content/drive/MyDrive/Colab Notebooks/Corpus/...
2,"Un hombre de 57 años, sin antecedentes de insu...",/content/drive/MyDrive/Colab Notebooks/Corpus/...
3,"Una niña de 55 días, por lo demás sana, con la...",/content/drive/MyDrive/Colab Notebooks/Corpus/...


In [ ]:
print(len(df))

14972


Cargamos el gazetteer proporcionado por el TEMU-BSC en la tarea compartida de PROFNER

In [ ]:
# Cargamos un gazetteer para reconocer en qué historias aparecen profesiones
gazetteer = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/profner-gazetteer.tsv', sep='\t',
                   names=['profesion', 'explicacion'], quoting=3, on_bad_lines='skip')

# Nos quedamos con la columna con las profesiones y eliminamos las dos primeras filas
gazetteer = pd.DataFrame(gazetteer.iloc[2:, 0])

# Vemos el aspecto del df
gazetteer.head(5)
print(len(gazetteer))

25252


# Identificación de notas susceptibles de contener menciones de ocupación

## Aplicamos el Gazetteer

Consideramos tres enfoques:

1.   Quedarnos con las cuatro primeras palabras del gazetteer
2.   Quedarnos sólo con la primera palabra del gazetteer
3.   Quedarnos con una palabra del gazetter y aplicar stemming

Nos vamos a quedar con el tercer enfoque



---



### Opción 1: Primeras cuatro palabras del gazetteer.

No usamos finalmente esta opción

In [ ]:
# Nos quedamos únicamente con las primeras cuatro palabras del gazetteer
gazetteer["recortado"] = gazetteer["profesion"].str.split(' ').str[0:4]
gazetteer["recortado"] = [' '.join(map(str, l)) for l in gazetteer['recortado']]
# Nos quedamos únicamente con las apariciones únicas
lista = gazetteer.recortado.unique()
len(lista)

# Si quisiéramos escapar caracteres especiales
# lista2 = '|'.join(re.escape(x) for x in lista)

# Eliminamos caracteres especiales (nos quedamos únicamente con alfanuméricos)
prueba = [re.sub(r'[^\w]', ' ', _) for _ in lista]
#prueba = [re.sub(r'[^\w]', ' ', _) for _ in lista[1:100]]
print(len(prueba))

# Transformamos en cadena de caracteres para hacer la búsqueda
prueba = '|'.join(prueba)
# https://stackoverflow.com/questions/68382051/use-str-contains-only-with-whole-words-using-pandas
# Para exact match
prueba = '\\b(' + prueba + ')\\b'
# Eliminamos múltiples espacios en blanco por un espacio
prueba =' '.join(prueba.split())

21077




---



### Opción 2: Primera palabra del gazetteer.

No usamos finalmente esta opción

In [ ]:
# Nos quedamos únicamente con la primera palabra del gazetteer
gazetteer["recortado"] = gazetteer["profesion"].str.split(' ').str[0:1]
gazetteer["recortado"] = [' '.join(map(str, l)) for l in gazetteer['recortado']]
# Nos quedamos únicamente con las apariciones únicas
lista = gazetteer.recortado.unique()
len(lista)

# Si quisiéramos escapar caracteres especiales
# lista2 = '|'.join(re.escape(x) for x in lista)

# Eliminamos caracteres especiales (nos quedamos únicamente con alfanuméricos)
prueba = [re.sub(r'[^\w]', ' ', _) for _ in lista]
print(len(prueba))

# Eliminamos palabras comúnes que podrían introducir errores
bannedWord = ["A", "au", "bb", "El", "el", "ex", "La", "Las", "la", "v", "Aquellos", "Aquellas"] # También quitar "estudio", "servicio", "Acompañantes", "acompañante"]
prueba = [word for word in prueba if word not in bannedWord]

print(len(prueba))


# Transformamos en cadena de caracteres para hacer la búsqueda
prueba = '|'.join(prueba)
# https://stackoverflow.com/questions/68382051/use-str-contains-only-with-whole-words-using-pandas
# Para exact match
prueba = '\\b(' + prueba + ')\\b'

# Eliminamos los espacios
prueba = prueba.replace(" ", "")

4305
4294


In [ ]:
# Mostramos el texto de búsqueda
prueba

'\\b(Acompañantes|Administradores|Agente|Agentes|Anestesistas|Archiveros|Asistentes|Audiólogos|Auxiliares|Ayudante|Bibliotecaria|Bibliotecarias|Bibliotecario|Bibliotecóloga|Bibliotecólogas|Bibliotecólogo|Bibliotecólogos|Bioeticistas|Brigada|Brigadas|CDO|CIO|CISO|CTO|Capellanes|Chaperones|Cirujanos|Clínicos|Cuerpo|Cuerpos|Cuidador|Cuidadores|Curas|Dentistas|Despachador|Directores|Diáconos|Docente|Docentes|Doulas|Educador|Educadores|Ejecutivos|Enfermera|Enfermeras|Enfermero|Enfermeros|Especialista|Especialistas|Examinadores|Experto|Farmacéuticos|Feldshers|Fisiatras|Funcionários|Gerente|Gestor|Gestores|Guarda|Guardianes|Histopatólogos|Imames|Individuos|Indivíduos|Maestros|Marines|Miembros|Mujeres|Médico|Médicos|NVOCC|Nurses|Oculistas|Odontólogos|Oficiales|Operador|Operadores|Orientadores|Ortodoncista|Ortopedistas|Paramédicos|Pastores|Peluquero|Personal|Personas|Policia|Policiais|Practicantes|Primer|Primeros|Profesionales|Profesionales|Profesor|Profesora|Profesores|Protesistas|Proveedores|

In [ ]:
# Usamos match
df['notasArevisar'] = df["texto"].str.match(prueba, na=False, case = False)
df.notasArevisar.value_counts()

False    5660
True     1564
Name: notasArevisar, dtype: int64

In [ ]:
df[df['notasArevisar']==True]

,texto,fichero,notasArevisar
8,"Mujer de 73 años, sin antecedentes personales ...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,True
18,Mujer de 29 años con antecedentes de ulcus duo...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,True
22,"Mujer de 51 años, monorrena derecha, incluída ...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,True
23,Mujer de 42 años en el momento de someterse a ...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,True
24,Mujer de 56 años que consulta al Servicio de U...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,True
...,...,...,...
5646,TÉCNICA: Se realiza videofluoroscopia con bolo...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,True
5667,Mujer 46 años.\nProfesora.\nAlergia a PNC.\nVi...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,True
5670,"Mujer de 71 años, diagnosticada mediante ecoca...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,True
6116,"Mujer de 34 años de edad, actualmente gestante...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,True




---



### Opción 3: primera palabra del gazetteer con stemming. Usamos esta opción

Opción finalmente usada. A destacar que aplicamos stemming sobre el gazetteer y sobre la nota clínica sin embargo el match lo hacemos entre el gazetteer stemmatizado y la nota clínica no stematizada. Con las pruebas realizadas, vemos que obtenemos el mismo resultado aplicando el gazetteer sobre la columna original o con stemming

In [ ]:
 # Cargamos el stemmer
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

# Nos quedamos únicamente con la primera palabra del gazetteer
gazetteer["recortado"] = gazetteer["profesion"].str.split(' ').str[0:1]
gazetteer["recortado"] = [' '.join(map(str, l)) for l in gazetteer['recortado']]
# Nos quedamos únicamente con las apariciones únicas
lista = gazetteer.recortado.unique()
len(lista)

# Si quisiéramos escapar caracteres especiales
# lista2 = '|'.join(re.escape(x) for x in lista)

# Eliminamos caracteres especiales (nos quedamos únicamente con alfanuméricos)
prueba = [re.sub(r'[^\w]', ' ', _) for _ in lista]
print(len(prueba))

# Eliminamos palabras comúnes que podrían introducir errores, mediante el empleo de reglas
bannedWord = ["A", "au", "bb", "El", "el", "ex", "La", "Las", "la", "las", "v", "control", "Aquellos", "Aquellas"] # También quitar "estudio", "servicio", "Acompañantes", "acompañante"]
prueba = [word for word in prueba if word not in bannedWord]
print("palabras después de aplicar bannerWord(stopwords)", len(prueba))

prueba = [stemmer.stem(word) for word in prueba]
prueba = list(dict.fromkeys(prueba))

print("palabras después de stemming", len(prueba))

bannedWord2 = ["Mujer", "Mujeres", "mujer", "negro", "nivel", "izquierda", "pequeño", "como", "rango", "otra", "nuevo", "grado", "tratamiento", "como", "mural", "el", "las", "con", "gran", "son", "ocular", "mas", "Test", "aplic", "control"] 

# Aplicamos un segundo filtro de palabras que pueden introducir errores
prueba = [word for word in prueba if word not in bannedWord2]
print("palabras después de stopwords2", len(prueba))

#prueba = [stemmer.stem(plural) for plural in prueba]
#prueba = [" ".join([stem(word) for word in sentence.split(" ")]) for sentence in prueba]

# Concatenamos todos los conceptos del gazetteer para hacer la búsqueda separando por |
prueba = '|'.join(prueba)

# https://stackoverflow.com/questions/68382051/use-str-contains-only-with-whole-words-using-pandas
# Para exact match
prueba = '\\b(' + prueba + ')\\b'

# Eliminamos los espacios
prueba = prueba.replace(" ", "")
prueba

4305
palabras después de aplicar bannerWord(stopwords) 4293
palabras después de stemming 3180
palabras después de stopwords2 3167


'\\b(acompañ|administr|agent|anestes|archiver|asistent|audiolog|auxiliar|ayud|bibliotecari|bibliotecolog|bioetic|brig|cdo|cio|cis|cto|capellan|chaperon|cirujan|clinic|cuerp|cuidador|cur|dentist|despach|director|diacon|docent|doul|educ|ejecut|enfermer|especial|examin|expert|farmaceut|feldshers|fisiatr|funcionari|gerent|gestor|guard|guardian|histopatolog|imam|individu|maestr|marin|miembr|medic|nvocc|nurses|ocul|odontolog|oficial|oper|orient|ortodonc|ortoped|paramed|pastor|peluquer|personal|person|polici|policiais|practic|prim|primer|profesional|profesionales|profesor|protes|proveedor|rabin|recepcion|respons|sacerdot|secretari|socorr|submariner|sindic|tecnolog|trabaj|tecnic|abacer|abad|abades|abander|abarroter|abastecedor|abatan|abat|abejer|ablak|abland|abog|abort|abrevi|abridg|abridor|abstraccion|abstractor|acab|academ|accesori|accesor|accion|accompanyist|aceit|acemiler|aceton|acical|acidif|acomod|acondicion|acordeon|acrobat|activ|actor|actriz|actuacion|actuari|acuchill|acupuntor|acupunt

Aplicamos stemming. Creamos una columna en la que al texto de la historia se haya aplicado stemming.

In [ ]:
df["stemmed"] = [stemmer.stem(word) for word in df["texto"]]

Hacemos uso de la función match para comprobar si una historia contiene el término del gazetteer. Esto lo deberíamos haber aplicado sobre la columna stemmed

In [ ]:
# Ponemos case = False para que el resultado no varíe en función de si el texto contiene o no mayúsculas 
df['match'] = df["texto"].str.match(prueba, na=False, case = False)
# Se debe aplicar sobre el stemmed
#df['match'] = df["stemmed"].str.match(prueba, na=False, case = False)

# Vemos los valores que contienen el gazetteer
df[df.match == True]

,texto,fichero,stemmed,match
1251,Menor de 4 años con anemia Blackfan Diamond re...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,menor de 4 años con anemia blackfan diamond re...,True
1252,Menor de 5 años con leucemia linfoblástica agu...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,menor de 5 años con leucemia linfoblastica agu...,True
6823,"AMO, 46 años, sexo masculino, procedente del m...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,"amo, 46 años, sexo masculino, procedente del m...",True
7821,"Otro de los casos es el de un paciente varón, ...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,"otro de los casos es el de un paciente varon, ...",True
10897,"Primer gemelo, producto de primera gestación d...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,"primer gemelo, producto de primera gestacion d...",True
11073,Primer hijo de padres sanos y no consanguíneos...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,primer hijo de padres sanos y no consanguineos...,True
13864,"Apicultor de 58 años, sin antecedentes de inte...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,"apicultor de 58 años, sin antecedentes de inte...",True
14370,"AMO, 46 años, sexo masculino, procedente del m...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,"amo, 46 años, sexo masculino, procedente del m...",True


In [ ]:
print(df["match"].value_counts())

False    14964
True         8
Name: match, dtype: int64


## Aplicamos las reglas


Creamos una columna que identifica todas las apariciones de los strings relacionados con el trabajo

In [ ]:
# En primer lugar comprobamos si la columna de la izquierda contiene palabras relacionadas con trabajo, ocupación etc
df["deteccion"] = df['texto'].str.contains('trabaj|ocupacion|profesion', case=False, regex=True)
print(df.deteccion.value_counts())

False    13710
True      1262
Name: deteccion, dtype: int64


In [ ]:
# Comprobamos como queda el df
df[df.deteccion == True]

,texto,fichero,stemmed,match,deteccion
0,"El 2 de febrero de 2020, una mujer de 28 años ...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,"el 2 de febrero de 2020, una mujer de 28 años ...",False,True
1,Ingresó en nuestro hospital un hombre de 23 añ...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,ingreso en nuestro hospital un hombre de 23 añ...,False,True
28,"﻿Una mujer de 31 años, por lo demás sana, acud...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,"﻿una mujer de 31 años, por lo demas sana, acud...",False,True
39,"﻿En abril de 2020, un hombre japonés de 57 año...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,"﻿en abril de 2020, un hombre japones de 57 año...",False,True
44,﻿Una mujer de 20 años embarazada secundigesta ...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,﻿una mujer de 20 años embarazada secundigesta ...,False,True
...,...,...,...,...,...
14871,Presentamos el caso de una paciente mujer de 1...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,presentamos el caso de una paciente mujer de 1...,False,True
14893,Niño de 9 años con faringitis desde hace 7 día...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,niño de 9 años con faringitis desde hace 7 dia...,False,True
14895,Se trata de un varón de 66 años con antecedent...,/content/drive/MyDrive/Colab Notebooks/Corpus/...,se trata de un varon de 66 años con antecedent...,False,True
14927,"Paciente de 42 años, casada, unigesta (gemelar...",/content/drive/MyDrive/Colab Notebooks/Corpus/...,"paciente de 42 años, casada, unigesta (gemelar...",False,True


Finalmente las notas a revisar son aquellas que, o bien han sido detectadas por el gazetteer o por los strings relacionados con trabajo

In [ ]:
# Notas candidatas de revisión
notas_revisar = df[(df['match'] == True)|(df['deteccion'] == True)]
print("Número notas candidatas: ", len(df[(df['match'] == True)|(df['deteccion'] == True)]))
print(len(notas_revisar))

# Usamos el codigo descrito en https://stackoverflow.com/questions/8384737/extract-file-name-from-path-no-matter-what-the-os-path-format
# Eliminamos la ruta para comparar el nombre de las notas a revisar con las presentes en el conjunto de test de MEDDOPROF
notas_revisar['fichero'] = notas_revisar.fichero.apply(lambda x: os.path.splitext(os.path.basename(x))[0])
#notas_revisar = notas_revisar[["texto", "fichero"]]
notas_revisar

Número notas candidatas:  1269
1269


<ipython-input-71-18b0fbfe04e8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notas_revisar['fichero'] = notas_revisar.fichero.apply(lambda x: os.path.splitext(os.path.basename(x))[0])


,texto,fichero,stemmed,match,deteccion
0,"El 2 de febrero de 2020, una mujer de 28 años ...",32119083_ES,"el 2 de febrero de 2020, una mujer de 28 años ...",False,True
1,Ingresó en nuestro hospital un hombre de 23 añ...,32168162_ES,ingreso en nuestro hospital un hombre de 23 añ...,False,True
28,"﻿Una mujer de 31 años, por lo demás sana, acud...",32426200_ES,"﻿una mujer de 31 años, por lo demas sana, acud...",False,True
39,"﻿En abril de 2020, un hombre japonés de 57 año...",32611659_ES,"﻿en abril de 2020, un hombre japones de 57 año...",False,True
44,﻿Una mujer de 20 años embarazada secundigesta ...,32614251_ES,﻿una mujer de 20 años embarazada secundigesta ...,False,True
...,...,...,...,...,...
14871,Presentamos el caso de una paciente mujer de 1...,cc_otorrinolaringologia44,presentamos el caso de una paciente mujer de 1...,False,True
14893,Niño de 9 años con faringitis desde hace 7 día...,es-S0210-56912007000200007-3,niño de 9 años con faringitis desde hace 7 dia...,False,True
14895,Se trata de un varón de 66 años con antecedent...,es-S0210-56912009000900008-1,se trata de un varon de 66 años con antecedent...,False,True
14927,"Paciente de 42 años, casada, unigesta (gemelar...",es-S0376-78922009000300005-1,"paciente de 42 años, casada, unigesta (gemelar...",False,True


# Eliminación de notas no utilizables

### Eliminamos notas duplicadas

Las notas duplicadas surgen por dos motivos:

1. Hay dos notas realmente duplicadas en el conjunto de datos
2. En el corpus hay una carpeta llamada background que contiene notas para complementar a los otros conjuntos. En algunos corpus las notas de background contienen parte de las notas de test (duplicados)

In [ ]:
# Eliminamos duplicados
notas_revisar[notas_revisar.duplicated(keep=False)]

,texto,fichero,stemmed,match,deteccion
0,"El 2 de febrero de 2020, una mujer de 28 años ...",32119083_ES,"el 2 de febrero de 2020, una mujer de 28 años ...",False,True
1,Ingresó en nuestro hospital un hombre de 23 añ...,32168162_ES,ingreso en nuestro hospital un hombre de 23 añ...,False,True
28,"﻿Una mujer de 31 años, por lo demás sana, acud...",32426200_ES,"﻿una mujer de 31 años, por lo demas sana, acud...",False,True
39,"﻿En abril de 2020, un hombre japonés de 57 año...",32611659_ES,"﻿en abril de 2020, un hombre japones de 57 año...",False,True
44,﻿Una mujer de 20 años embarazada secundigesta ...,32614251_ES,﻿una mujer de 20 años embarazada secundigesta ...,False,True
...,...,...,...,...,...
14843,Anamnesis\nSe presenta el caso de un paciente ...,cc_onco797,anamnesis\nse presenta el caso de un paciente ...,False,True
14844,"Anamnesis\nHombre de 75 años, sin alergias med...",cc_onco800,"anamnesis\nhombre de 75 años, sin alergias med...",False,True
14851,Anamnesis\nAntecedentes familiares: tío matern...,cc_onco922,anamnesis\nantecedentes familiares: tio matern...,False,True
14855,"Anamnesis\nMujer de 54 años, sin antecedentes ...",cc_onco95,"anamnesis\nmujer de 54 años, sin antecedentes ...",False,True


In [ ]:
print("Dimensión total:", len(notas_revisar))
notas_revisar = notas_revisar.drop_duplicates()
print("Dimensión sin duplicados:", len(notas_revisar))

Dimensión total: 1269
Dimensión sin duplicados: 1109


In [ ]:
# Vemos que hay duplicados con pequeñas diferencias (por ejemplo, salto de línea. Esto lo vemos para el archivo S1130-01082008000900011-1 que está a la vez en background y en test)
notas_revisar.fichero.value_counts()

32119083_ES                         1
caso_clinico_urologia117            1
caso_clinico_urologia434            1
caso_clinico_urologia373            1
caso_clinico_urologia321            1
                                   ..
caso_clinico_medicina_interna793    1
caso_clinico_medicina_interna881    1
caso_clinico_medicina_interna888    1
caso_clinico_medicina_interna890    1
es-S0465-546X2009000300008-1        1
Name: fichero, Length: 1109, dtype: int64



---



### Eliminamos notas que se encuenten en MEDDOPROF

Cargamos el nombre de las notas del conjunto de entrenamiento/test de MEDDOPROF. Al estar ya anotadas por expertos, no las necesitamos anotar nosotros. El objetivo es ampliar este conjunto con notas nuevas

In [ ]:
# Directorio con las notas de MEDDOPROF
DATA_DIR_TRAIN = "/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Original/Train/"
data = load_files(DATA_DIR_TRAIN, encoding="utf-8", decode_error="replace", shuffle=False)
data_train = pd.DataFrame(data.data, columns = ['texto'])
data_train['fichero'] = data.filenames

# Nos quedamos únicamente con los ficheros de estudio en base a como se organiza el corpus en carpetas
data_train = data_train[(data_train.fichero.str.contains('.txt', case=False, regex=False)) & (data_train.fichero.str.contains('task1', case=False, regex=False))]

#df["deteccion"] = df['texto'].str.contains('trabaj|ocupacion|profesion', case=False, regex=True)

#Eliminamos la ruta para comparar el nombre de las notas a revisar con las presentes en el conjunto de test de MEDDOPROF
data_train['fichero'] = data_train["fichero"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
data_train.head(5)

,texto,fichero
1,Un paciente de unos cincuenta años sin anteced...,32247016_ES
3,"﻿Una mujer de 27 años sin antecedentes, reside...",32294273_ES
5,﻿Mujer de 66 años que acudió refiriendo dificu...,32399950_ES
7,﻿El 28 de febrero de 2020 ingresó en urgencias...,32417987_ES
9,"﻿Una mujer de 31 años, por lo demás sana, acud...",32426200_ES


In [ ]:
print("Notas en el conjunto de entrenamiento MEDDOPROF:", len(data_train))
print("Notas en el corpus de estudio:", len(notas_revisar))
print("Notas en ambos conjuntos:", len(data_train.merge(notas_revisar, how = 'inner', indicator = True, on = ['fichero','texto'])))
print("Notas totales a seleccionar:", len(notas_revisar)-len(data_train.merge(notas_revisar, how = 'inner', indicator = True, on = ['fichero','texto'])))

Notas en el conjunto de entrenamiento MEDDOPROF: 1500
Notas en el corpus de estudio: 185
Notas en ambos conjuntos: 0
Notas totales a seleccionar: 185


In [ ]:
# Nos quedamos únicamente con las notas que no estén en el conjunto data_train (según la columna fichero)
notas_resultantes = notas_revisar[~notas_revisar['fichero'].isin(data_train["fichero"])]
print(len(notas_resultantes))

159


Cargamos el nombre de las notas del conjunto de test de MEDDOPROF. Ninguna nota del conjunto de test debe estar en las notas candidatas a anotar. Para ello comprobamos el contenido de la nota y el nombre del fichero

In [ ]:
DATA_DIR_TEST = "/content/drive/MyDrive/Colab Notebooks/Corpus/MEDDOPROF/Original/Test/"
data = load_files(DATA_DIR_TEST, encoding="utf-8", decode_error="replace", shuffle=False)

data_test = pd.DataFrame(data.data, columns = ['texto'])
data_test['fichero'] = data.filenames

# Nos quedamos únicamente con los ficheros de estudio en base a como se organiza el corpus en carpetas
data_test = data_test[(data_test.fichero.str.contains('.txt', case=False, regex=False)) & (data_test.fichero.str.contains('ner', case=False, regex=False))]

#df["deteccion"] = df['texto'].str.contains('trabaj|ocupacion|profesion', case=False, regex=True)

#Eliminamos la ruta para comparar el nombre de las notas a revisar con las presentes en el conjunto de test de MEDDOPROF
data_test['fichero'] = data_test["fichero"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
data_test.head(5)

,texto,fichero
689,﻿Un hombre de 36 años llegó al servicio de urg...,32423911_ES
691,"﻿El señor G, un hombre de 43 años sin antecede...",32605766_ES
693,﻿Una mujer de 20 años embarazada secundigesta ...,32614251_ES
695,Nombre: Miriam .\nApellidos: Aguilar Mora.\nC...,S0004-06142009000100010-2
697,Nombre: Paula.\nApellidos: De la Cruz Lorenzo....,S0034-98872006000200011-1


In [ ]:
notas_resultantes = notas_resultantes[~notas_resultantes['fichero'].isin(data_test["fichero"])]
print(len(notas_resultantes))

155


In [ ]:
# Vemos finalmente el conjunto de datos
notas_resultantes.head(5)
notas_resultantes = notas_resultantes.drop_duplicates()
notas_resultantes

,texto,fichero,stemmed,match,deteccion
86,Presentamos el caso de un paciente varón de 51...,S0004-06142007000700009-1,presentamos el caso de un paciente varon de 51...,False,True
110,Paciente de sexo femenino y 33 años de edad qu...,S0004-06142008000200037-1,paciente de sexo femenino y 33 años de edad qu...,False,True
174,"La historia corresponde a un varón de 30 años,...",S0004-06142009000800009-1,"la historia corresponde a un varon de 30 años,...",False,True
267,"Paciente de 45 años, con antecedentes de hiper...",S0034-98872007000900012-1,"paciente de 45 años, con antecedentes de hiper...",False,True
325,"Primigesta de 15 años de edad, con gestación d...",S0034-98872009000900010-1,"primigesta de 15 años de edad, con gestacion d...",False,True
...,...,...,...,...,...
3427,"Mujer de 68 años, como antecedentes personales...",S1130-01082007000700011-2,"mujer de 68 años, como antecedentes personales...",False,True
3523,Varón de 46 años sin antecedentes de interés q...,S1137-66272016000200015-1,varon de 46 años sin antecedentes de interes q...,False,True
3551,Mujer de 40 años que acude a la consulta de At...,S1699-695X2015000200010-1,mujer de 40 años que acude a la consulta de at...,False,True
3552,Paciente de 59 años que refiere dificultad par...,S1699-695X2015000300010-1,paciente de 59 años que refiere dificultad par...,False,True


Nos quedamos únicamente con el nombre del archivo

In [ ]:
notas_resultantes_indices = notas_resultantes.fichero.drop_duplicates()
print(len(notas_resultantes_indices))
notas_resultantes_indices

151


86      S0004-06142007000700009-1
110     S0004-06142008000200037-1
174     S0004-06142009000800009-1
267     S0034-98872007000900012-1
325     S0034-98872009000900010-1
                  ...            
3427    S1130-01082007000700011-2
3523    S1137-66272016000200015-1
3551    S1699-695X2015000200010-1
3552    S1699-695X2015000300010-1
3555    S1887-85712012000400006-1
Name: fichero, Length: 151, dtype: object

Guardamos tanto las notas resultantes (con el contenido) como el nombre de la nota (sin extensión a anotar)

Las notas resultantes de este script nos sirven a modo de comprobación pero no las vamos a usar

In [ ]:
#Guardamos las notas
#notas_resultantes.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/notas_BARR2.txt", sep = "\t", index = False)
#notas_resultantes.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/notas_CANTEMIST.txt", sep = "\t", index = False)
#notas_resultantes.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/notas_CODIESP.txt", sep = "\t", index = False)
#notas_resultantes.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/notas_IULA.txt", sep = "\t", index = False)
#notas_resultantes.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/notas_LIVINGNER.txt", sep = "\t", index = False)
#notas_resultantes.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/notas_MEDDOCAN.txt", sep = "\t", index = False)
#notas_resultantes.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/notas_NUBEs.txt", sep = "\t", index = False)
#notas_resultantes.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/notas_PHARMACONER.txt", sep = "\t", index = False)
#notas_resultantes.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/notas_PROFNER.txt", sep = "\t", index = False)

Los índices de las notas es el output que vamos a utilizar en el script `ExtraccionNotas.ipynb`

In [ ]:
#Guardamos los indices de las notas
#notas_resultantes_indices.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/indices_BARR2.txt", sep = "\t", index = False)
#notas_resultantes_indices.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/indices_CANTEMIST.txt", sep = "\t", index = False)
#notas_resultantes_indices.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/indices_CODIESP.txt", sep = "\t", index = False)
#notas_resultantes_indices.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/indices_IULA.txt", sep = "\t", index = False)
#notas_resultantes_indices.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/indices_LIVINGNER.txt", sep = "\t", index = False)
#notas_resultantes_indices.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/indices_MEDDOCAN.txt", sep = "\t", index = False)
#notas_resultantes_indices.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/indices_NUBEs.txt", sep = "\t", index = False)
#notas_resultantes_indices.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/indices_PHARMACONER.txt", sep = "\t", index = False)
#notas_resultantes_indices.to_csv("/content/drive/MyDrive/Colab Notebooks/anotar/indices_PROFNER.txt", sep = "\t", index = False)

# Referencias
[1] https://spacy.io/usage/rule-based-matching

[2] https://stackoverflow.com/questions/29771168/how-to-remove-words-from-a-list-in-python

[3] https://stackoverflow.com/questions/875968/how-to-remove-symbols-from-a-string-with-python